In [24]:
from collections import Counter
from collections import OrderedDict
import re
from __future__ import division
import numpy as np
import string
import time

In [2]:
# pos_tags = [u'CC', u'CD', u'DT', u'EX', u'FW', u'IN', u'JJ', u'JJR', u'JJS', u'LS', u'MD',
#             u'NN', u'NP', u'NPS', u'NNS', u'PDT', u'POS', u'PP', u'PP$', u'RB', u'RBR',
#             u'RBS', u'RP', u'SYM', u'TO', u'UH', u'VB', u'VBD', u'VBG', u'VBN', u'VBP', u'VBZ',
#             u'WDT', u'WP', u'WP$', u'WRB']

# with open("POS.train", 'r') as train: 
#     lines = train.readlines()
# #     pos_tags_single = re.findall('(/[A-Z]{2,3}\$*)', text)
# #     pos_tags = re.findall('(/[A-Z]{2,3}\$*\|*[A-Z]{2,3}\$*)', text)
# # pos_tags = set(pos_tags).union(set(pos_tags_single))

# # pos_tags = [x[1:] for x in pos_tags]
# print(pos_tags,len(pos_tags))

In [3]:
# # c(wi,ti)
# c_wi_ti = Counter() # in the order wi/ti
# c_ti = Counter()
# c_ti_tim1 = Counter() # in order t_i/t_{i-1}
# num_lines = 0
# with open("POS.train", 'r') as train:
#     for line in train:
#         num_lines += 1
#         c_wi_ti.update(Counter(line[:-2].split(' ')))
#         # currently, this gets all things after / even if it's word1\/word2 which i dont want (such as Cities\/ABC/NP)
#         pos_order = re.findall('(/[A-Z]{2,3}\$*)', line) # finds things like /DT
#         c_ti.update(Counter(pos_order))
#         for ind, pos in enumerate(pos_order):
#             if ind == 0:
#                 # pos[1:] bc it starts w/ "/"
#                 c_ti_tim1.update(Counter([pos[1:] + '/<s>']))
#             else:
#                 c_ti_tim1.update(Counter([pos[1:] + pos_order[ind-1]]))
# c_ti['/<s>'] = num_lines
# del c_ti['/ABC'] # GET RID OF this mistake until I fix my regex

In [4]:
# c_ti

In [5]:
# # this does not include punctuation... should I? It seems like a hard rule lol
# import time
# start = time.time()
# p_wi_ti = {}
# # for POS, POS_count in c_ti.items():
# #     for key, val in c_wi_ti.items():
# #         if POS in key:
# #             p_wi_ti[key] = val / POS_count
# for key, val in c_wi_ti.items():
#     pos = '/' + key.split('/')[1]
#     try:
#         p_wi_ti[key] = val / c_ti[pos]
#     except: # need to fix this but for now make it .001
#         p_wi_ti[key] = .001
# # print(len(p_wi_ti), p_wi_ti['the/DT'], time.time()-start)


In [6]:
# p_ti_tim1 = {}
# # <s>
# for key, val in c_ti_tim1.items():
#     pos = '/' + key.split('/')[-1] # we want total count of this pos for denominator
#     try:
#         p_ti_tim1[key] = val / c_ti[pos]
#     except: # need to fix this but for now make it .001
#         p_ti_tim1[key] = .001



In [25]:

filename = "POS.train"
def create_prob_dict(filename):
    unigram = []
    bigram = []
    bigram_count= {}
    i = 0

    with open (filename,'r') as training:
        lines = training.readlines()
        start = len(lines)

        for line in lines:
            line = line.split()
            line.append('start')
            for item in line:
                tag = item.split('/')[-1]
                # if it's an ambiguous tag we get the second one
                if "|" in tag:
                    tag = tag.split("|")[-1]

                unigram.append(tag)

        for word in unigram:

            if i < len(unigram)-1:
                bigram.append(unigram[i] + ' '+unigram[i+1])
                i+=1

        new_bigram = [x for x in bigram if not x.endswith("start")]

        for item in new_bigram:       
            if item not in bigram_count:
                bigram_count[item]=1
            else:
                bigram_count[item]+=1

    # tag_frequency, pos_word_prob 
    pos = []
    with open (filename,'r') as training:
        string_r = training.read()

        splited_string = string_r.split()

        for item in splited_string:
            pos.append(item.split('/')[-1]) #Choose the last one when there are multiple slashes.

        tag_frequency = Counter(pos)
        tag_frequency.update({'start':start})

        pos_word_count = Counter(splited_string)
        pos_word_prob = pos_word_count

        for item in pos_word_count:
            pos_word_prob[item] = pos_word_count[item]/float(tag_frequency[item.split('/')[-1]])

        # bigram_prob
    bigram_prob = {}
    for key,value in bigram_count.items():
        bigram_prob[key]=value/float(tag_frequency[key.split()[0]])
        
    return unigram, bigram_prob, lines

In [27]:
# unigram, bigram_prob, lines = create_prob_dict(filename)
bigram_prob

{'VB $': 0.008693089730366877,
 'VBG VBN': 0.026621866115275266,
 "'' ``": 0.012979683972911963,
 'DT JJS': 0.008354968137833372,
 'VBN VBN': 0.030763314747164008,
 'CD NNS': 0.16135639508420574,
 'VBZ JJS': 0.0009300595238095238,
 'VBG VBG': 0.004393900232618248,
 ', NN': 0.04636683869910707,
 'VBN VBD': 0.0030763314747164006,
 ', WDT': 0.03768329646923896,
 'WP ``': 0.0016286644951140066,
 'WDT NNS': 0.017436791630340016,
 'VBG VBZ': 0.001809253036960455,
 'JJS DT': 0.014925373134328358,
 'VBN VBP': 0.0019227071716977505,
 'VB NPS': 0.0004420215117135701,
 'VBG VBP': 0.0005169294391315585,
 'VBD VBN': 0.09473007712082263,
 'MD RBR': 0.00238758456028651,
 'VBG :': 0.001809253036960455,
 'WRB RB': 0.04356435643564356,
 'FW VBZ': 0.04838709677419355,
 'PP MD': 0.1213218650973291,
 'TO NPS': 0.0003513086246267346,
 'DT NN': 0.46990795374085437,
 'WP RB': 0.030944625407166124,
 "NN ''": 0.005998108970570855,
 'RB WDT': 0.000640861317610869,
 '`` EX': 0.018753447324875896,
 'POS RBS': 0.00

In [28]:
all_tags = []
for key in bigram_prob.keys(): # in order i-1 i
    all_tags.extend(key.split())

# store unique tags
tag_list = list(set(all_tags))


In [29]:
# create ordered word mapping to observation matrix (mainly for testin)
def create_word_mapping(line_list):
    words = []
    ind = 0
    word_map = OrderedDict()
    for line in lines:
        for pair in line.split():
            word = pair.split('/')[0]
            if word not in word_map:
                word_map[word] = ind
                ind += 1
    return word_map

word_map = create_word_mapping(lines)

In [30]:
word_map

OrderedDict([('Pierre', 0),
             ('Vinken', 1),
             (',', 2),
             ('61', 3),
             ('years', 4),
             ('old', 5),
             ('will', 6),
             ('join', 7),
             ('the', 8),
             ('board', 9),
             ('as', 10),
             ('a', 11),
             ('nonexecutive', 12),
             ('director', 13),
             ('Nov.', 14),
             ('29', 15),
             ('.', 16),
             ('Mr.', 17),
             ('is', 18),
             ('chairman', 19),
             ('of', 20),
             ('Elsevier', 21),
             ('N.V.', 22),
             ('Dutch', 23),
             ('publishing', 24),
             ('group', 25),
             ('Rudolph', 26),
             ('Agnew', 27),
             ('55', 28),
             ('and', 29),
             ('former', 30),
             ('Consolidated', 31),
             ('Gold', 32),
             ('Fields', 33),
             ('PLC', 34),
             ('was', 35),
             ('

In [31]:
trans_p = np.zeros(shape=(len(tag_list), len(tag_list)))
print(trans_p.shape, len(trans_p))

((46, 46), 46)


In [32]:
for col_ind, tag in enumerate(tag_list):
    for row_ind, cond_tag in enumerate(tag_list):

        ti_tim1 = cond_tag + ' ' + tag
        try:
            print('trans_p at {},{} is {} w/ val {}'.format(row_ind, col_ind,
                                                            ti_tim1, bigram_prob[ti_tim1]))
            trans_p[row_ind, col_ind] = bigram_prob[ti_tim1]
        except:
            pass
#             print('trans_p at {},{} is {} w/ val {}'.format(row_ind, col_ind,
#                                                             ti_tim1, 0))
        

trans_p at 0,0 is VBG VBG w/ val 0.00439390023262
trans_p at 1,0 is VBD VBG w/ val 0.0214652956298
trans_p at 2,0 is VB VBG w/ val 0.0163547959334
trans_p at 3,0 is POS VBG w/ val 0.00905797101449
trans_p at 4,0 is '' VBG w/ val 0.0129796839729
trans_p at 5,0 is VBP VBG w/ val 0.0819366852886
trans_p at 7,0 is JJ VBG w/ val 0.00358693600151
trans_p at 9,0 is VBZ VBG w/ val 0.0628720238095
trans_p at 10,0 is DT VBG w/ val 0.00882700023602
trans_p at 11,0 is PP VBG w/ val 0.0027161611589
trans_p at 12,0 is RP VBG w/ val 0.0214132762313
trans_p at 14,0 is NN VBG w/ val 0.00803687507387
trans_p at 15,0 is start VBG w/ val 0.0097
trans_p at 16,0 is ) VBG w/ val 0.0149253731343
trans_p at 17,0 is ( VBG w/ val 0.00779220779221
trans_p at 19,0 is , VBG w/ val 0.0460391578602
trans_p at 21,0 is TO VBG w/ val 0.00720182680485
trans_p at 23,0 is RB VBG w/ val 0.0298641374007
trans_p at 24,0 is NP VBG w/ val 0.00159476246433
trans_p at 25,0 is : VBG w/ val 0.0220152413209
trans_p at 26,0 is NNS VB

In [33]:
init_state = np.zeros(shape=(len(tag_list), 1))

In [34]:
for ind, tag in enumerate(tag_list):
    try:
#         print('init_state at {}, is {} w/ val {}'.format(ind, 'start' + " "+ tag,
#                                                          bigram_prob['start' + ' ' + tag]))       
        init_state[ind] = bigram_prob['start' + ' ' + tag]
    except:
        pass

In [35]:
class Decoder(object):
    def __init__(self, initialProb, transProb, obsProb):
        self.N = initialProb.shape[0]
        self.initialProb = initialProb
        self.transProb = transProb
        self.obsProb = obsProb
        assert self.initialProb.shape == (self.N, 1)
        assert self.transProb.shape == (self.N, self.N)
        assert self.obsProb.shape[0] == self.N

    def Obs(self, obs):
        return self.obsProb[:, obs, None]

    def Decode(self, obs):
        trellis = np.zeros((self.N, len(obs)))
        backpt = np.ones((self.N, len(obs)), 'int32') * -1
        
        # initialization
        # first row of trans matrix time first column of 
        # emission matrix
        trellis[:, 0] = np.squeeze(self.initialProb * self.Obs(obs[0]))

        for t in xrange(1, len(obs)):
            # trellis[:, t-1, None] is previous column of figure 10.9
            # self.Obs(obs[t]) is the t_th column of our p(w_i|t_i) matrix
            trellis[:, t] = (trellis[:, t-1, None].dot(self.Obs(obs[t]).T) * self.transProb).max(0)
            backpt[:, t] = (np.tile(trellis[:, t-1, None], [1, self.N]) * self.transProb).argmax(0)
#         print(backpt)
        # termination
        tokens = [trellis[:, -1].argmax()]
        for i in xrange(len(obs)-1, 0, -1):
            tokens.append(backpt[tokens[-1], i])
        return tokens[::-1]

# To do/questions
- create obs matrix and run decoder for each sentence,
    - for each sentence, get error??
- testing? p_wi_ti, p_ti_tim1, obs matrix and transition matrix should all be fixed from using the training data? This means I have to create obs matrix for the whole text in the training data?? So basically, in the training file for sentence one, I map each word to a state and in the text file, I find the state of a given word and use that? if a word is new??

In [36]:
o_mat = np.zeros(shape=(len(tag_list), len(word_map.keys())))

for col_ind, word in enumerate(word_map.keys()):
    for row_ind, pos in enumerate(tag_list):
        wi_ti = word + '/' + pos
        try:
#             print('obs_mat at {},{} is {} w/ val {}'.format(row_ind, col_map,
#                                                             wi_ti, pos_word_prob[wi_ti]))
            o_mat[row_ind, col_ind] = pos_word_prob[wi_ti]
        except:
#             print('obs_mat at {},{} is {} w/ val {}'.format(row_ind, col_ind,
#                                                             wi_ti, 0))       
            pass


In [37]:
def run_decoder(decoder, filename):
    with open(filename, 'r') as f:
        line_list = f.readlines()
    truth_counter = 0
    num_chances = 0
    num_correct = 0
    result_list = []
    for line in line_list[0:10]:
        sentence = []
        state = []
        for col_ind, txt in enumerate(line.split()):
            word = txt.split('/')[0]
    #         print('word:state is {}:{}'.format(word, word_map[word]))
            state.append(word_map[word])
            sentence.append(word)

        # run Decode method and get result
        states = decoder.Decode(np.array(state))
        result = np.array(tag_list)[states].tolist()

        resultTagged = zip(sentence,result)
        result_list.append(resultTagged)
    #     print(resultTagged)
    #     print(line)
    #     print('\n')
        for val in result:
            num_chances += 1
            if unigram[truth_counter] == 'start':
                truth_counter += 1
            if val == unigram[truth_counter]:
                num_correct += 1

            truth_counter += 1
    accuracy = num_correct / num_chances
    return accuracy, result_list

In [38]:
decoder = Decoder(initialProb=init_state, transProb=trans_p, obsProb=o_mat)
acc, rslt_list = run_decoder(decoder, filename)

In [39]:
print(acc)

# with open('POS.test.out', 'w') as outfile:
#     for sent in rslt_list:
#         for pair in sent:
#             outfile.write('%s/%s ' %pair)
#         outfile.write('\n')


0.0224215246637


In [23]:
rslt_list

[[('Pierre', 'VBG'),
  ('Vinken', 'VBG'),
  (',', 'VBG'),
  ('61', 'VBG'),
  ('years', 'VBG'),
  ('old', 'VBG'),
  (',', 'VBG'),
  ('will', 'VBG'),
  ('join', 'VBG'),
  ('the', 'VBG'),
  ('board', 'VBG'),
  ('as', 'VBG'),
  ('a', 'VBG'),
  ('nonexecutive', 'VBG'),
  ('director', 'VBG'),
  ('Nov.', 'VBG'),
  ('29', 'VBG'),
  ('.', 'VBG')],
 [('Mr.', 'VBG'),
  ('Vinken', 'VBG'),
  ('is', 'VBG'),
  ('chairman', 'VBG'),
  ('of', 'VBG'),
  ('Elsevier', 'VBG'),
  ('N.V.', 'VBG'),
  (',', 'VBG'),
  ('the', 'VBG'),
  ('Dutch', 'VBG'),
  ('publishing', 'VBG'),
  ('group', 'VBG'),
  ('.', 'VBG')],
 [('Rudolph', 'VBG'),
  ('Agnew', 'VBG'),
  (',', 'VBG'),
  ('55', 'VBG'),
  ('years', 'VBG'),
  ('old', 'VBG'),
  ('and', 'VBG'),
  ('former', 'VBG'),
  ('chairman', 'VBG'),
  ('of', 'VBG'),
  ('Consolidated', 'VBG'),
  ('Gold', 'VBG'),
  ('Fields', 'VBG'),
  ('PLC', 'VBG'),
  (',', 'VBG'),
  ('was', 'VBG'),
  ('named', 'VBG'),
  ('a', 'VBG'),
  ('nonexecutive', 'VBG'),
  ('director', 'VBG'),
  ('of',